In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:
correspondance = {
    "frouge": 0,
    "forange": 1,
    "fvert": 2,
    "stop": 3,
    "ceder": 4,
    "interdiction": 5,
    "danger": 6,
    "obligation": 7
}

In [ ]:
train_img_dir = "../../dataset/train/images"
train_lab_dir = "../../dataset/train/labels"

val_img_dir = "../../dataset/val/images"
val_lab_dir = "../../dataset/val/labels"

data_dir = "data"
data_train_file = "data/train.txt"
data_val_file = "data/val.txt"

train_img_dir_dest = "data/train/images"
train_lab_dir_dest = "data/train/labels"

val_img_dir_dest = "data/val/images"
val_lab_dir_dest = "data/val/labels"

In [ ]:
# si repertoire n'existe pas, on le cree
if not os.path.exists(train_img_dir_dest):
    os.makedirs(train_img_dir_dest)

if not os.path.exists(train_lab_dir_dest):
    os.makedirs(train_lab_dir_dest)

if not os.path.exists(val_img_dir_dest):
    os.makedirs(val_img_dir_dest)

if not os.path.exists(val_lab_dir_dest):
    os.makedirs(val_lab_dir_dest)


In [ ]:
img_size = 640

In [ ]:
def create(img_dir, lab_dir, img_dir_dest, lab_dir_dest, data_file):

    # file = open(data_file, 'w')
    for img in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img)
        lab_path = os.path.join(lab_dir, img[:-4] + ".csv")

        img_dest_path = os.path.join(img_dir_dest, img)
        lab_dest_path = os.path.join(lab_dir_dest, img[:-4] + ".txt")

        # redimensionnement de l'image
        img = cv2.imread(img_path)
        h, w, _ = img.shape
        img = cv2.resize(img, (img_size, img_size))

        # enregistrement de l'image
        cv2.imwrite(img_dest_path, img)

        # ecriture du chemin de l'image dans le fichier data
        # file.write(f"{img_dest_path} {lab_dest_path}\n")

        with open(lab_path, 'r') as f:
            lines = f.readlines()
            if "\n" in lines: lines.remove("\n")

        for line in lines:
            x1, y1, x2, y2 = map(int, line.split(",")[:4])
            type_panneau = line.split(",")[4].strip()
            if type_panneau == 'ff':
                continue

            # calcul des coordonnees du centre
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            width = x2 - x1
            height = y2 - y1

            # leur affecter leur changement induit du resize
            x_center = int(round(x_center / w * img_size))/img_size
            y_center = int(round(y_center / h * img_size))/img_size
            width = int(round(width / w * img_size))/img_size
            height = int(round(height / h * img_size))/img_size

            # afficher la box de l'image avec les labels modifiés
            # box = cv2.rectangle(img, (int(x_center - width / 2), int(y_center - height / 2)), (int(x_center + width / 2), int(y_center + height / 2)), (0, 255, 0), 2)
            # plt.imshow(cv2.cvtColor(box, cv2.COLOR_BGR2RGB))

            # enregistrement dans le fichier
            with open(lab_dest_path, 'a') as f:
                f.write(f"{correspondance[type_panneau]} {x_center} {y_center} {width} {height}\n")

    # file.close()
            

In [ ]:
# creation des fichiers pour le train
create(train_img_dir, train_lab_dir, train_img_dir_dest, train_lab_dir_dest, data_train_file)

# creation des fichiers pour le val
create(val_img_dir, val_lab_dir, val_img_dir_dest, val_lab_dir_dest, data_val_file)